In [1]:
import os
import pinecone
from os.path import join, dirname
from dotenv import load_dotenv

# load environment
env_filepath = os.path.join(dirname('__file__'), '.env')

load_dotenv(env_filepath)

pinecone_api_key = os.getenv("PINECONE_API_KEY")
env_name = "gcp-starter"

pinecone.init(api_key=pinecone_api_key, environment=env_name)

In [2]:
# Check pinecone-client compatibility
import pinecone.info
import console

version_info = pinecone.info.version()
server_version = ".".join(version_info.server.split(".")[:2])
print(f"Pinecone server version: {server_version}")
client_version = ".".join(version_info.client.split(".")[:2])
print(f"Pinecone client version: {client_version}")

assert client_version == server_version, "Pinecone server and client versions do not match. PLease  update the Pinecone client."

Pinecone server version: 2.0
Pinecone client version: 2.0


In [4]:
# Connect to Pinecone index
try:
    pinecone_index = pinecone.Index(os.getenv("PINECONE-INDEX_NAME"))
except Exception as e:
    print(e.message)

In [ ]:
# PDF Parsing
# Methodology taken from:
# https://towardsdatascience.com/extracting-text-from-pdf-files-with-python-a-comprehensive-guide-9fc4003d517
# Author: George Stavrakis

import PyPDF2
import pdfplumber
import pytesseract 
from PIL import Image
from pdf2image import convert_from_path
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

pdfs_dir_path = os.path.join(dirname('__file__'), 'pdfs')

# Step 1: Prelimary analysis of PDF documents to determine each PDF type:
# programatically generated; scanned images; or scanned documents with OCR.
# Use PDFMiner to take a PDF document object of multiple pages and analyse each page.
def convert_pdf_to_text(filepath):
    for page_number, page in enumerate(extract_pages(filepath)):
        for element in page:
            if isinstance(element, LTTextContainer):
                # Function to extract text from text block
                pass
            elif isinstance(element, LTFigure):
                # Function to convert PDF to image
                pass
            elif isinstance(element, LTRect):
                # Function to extract text with OCR
                pass
            else:
                console.warning("Encountered element of unknown type - unable to parse. Processing next element...")
        

                




def convert_pdf_to_text(filepath):
    with open(filepath, 'rb') as file:
        pdf = PdfFileReader(file)
        

def parse_pdfs(pdfs_dir_path):
    if not os.path.exists(pdfs_dir_path):
        raise FileNotFoundError(f"The directory {pdfs_dir_path} cannot be found.")

    for filename in os.listdir(pdfs_dir_path):
        if filename.lower().endswith('.pdf'):
            filepath = os.path.join(pdfs_dir_path, filename)

In [5]:
testpath = r"/Users/niallmcmanus/Documents/TrafficLight"
os.listdir(testpath)

['testing.ipynb', 'README.md', '.gitignore', 'pdfs', '.git']